In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 10:53:51 WARN Utils: Your hostname, sugab-archlinux resolves to a loopback address: 127.0.1.1; using 192.168.81.206 instead (on interface wlan0)
24/03/03 10:53:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 10:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Group By

In [2]:
df_green = spark.read.parquet("data/raw/green/*/*")

In [3]:
df_green.registerTempTable('green')

/home/sugab/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql(
    """
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
"""
)

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet("data/raw/yellow/*/*")
df_yellow.registerTempTable("yellow")

In [7]:
df_yellow_revenue = spark.sql(
    """
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
"""
)

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

## Join

### 2 Large Dataset

In [9]:
df_green_revenue = spark.read.parquet("data/report/revenue/green")
df_yellow_revenue = spark.read.parquet("data/report/revenue/yellow")

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  45|              NULL|                NULL| 732.4799999999997|                   42|
|2020-01-01 00:00:00|  49|            266.76|                  14|            185.65|                   10|
|2020-01-01 00:00:00|  59|50.900000000000006|                   3|              NULL|                 NULL|
|2020-01-01 00:00:00|  65|            199.49|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00|  74| 3

In [12]:
df_join.write.parquet("data/report/revenue/total", mode="overwrite")

## Large Dataset and Small Dataset

In [13]:
df_join = spark.read.parquet("data/report/revenue/total")
df_join.show(5)

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|             NULL|                NULL|1004.2999999999995|                   57|
|2020-01-01 00:00:00|   7|769.7299999999997|                  45| 455.1700000000002|                   38|
|2020-01-01 00:00:00|  12|             NULL|                NULL|             107.0|                    6|
|2020-01-01 00:00:00|  13|             NULL|                NULL|1214.7999999999993|                   56|
|2020-01-01 00:00:00|  24|             87.6|                   3| 754.9499999999997|                   45|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
only showing top 5 rows



In [17]:
df_zones = spark.read.parquet('zones/')
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [18]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)
df_result.show(5)

+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+-----------------+------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|             Zone|service_zone|
+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+-----------------+------------+
|2020-01-01 00:00:00|   4|             NULL|                NULL|1004.2999999999995|                   57|         4|Manhattan|    Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|   7|769.7299999999997|                  45| 455.1700000000002|                   38|         7|   Queens|          Astoria|   Boro Zone|
|2020-01-01 00:00:00|  12|             NULL|                NULL|             107.0|                    6|        12|Manhattan|     Battery Park| Yellow Zone|
|2020-01-01 00:00:00|  13|             NULL|  

In [19]:
df_result.drop("LocationID", "zone").write.parquet("tmp/revenue-zones")